# Requirments
- Python 3.11.9
    - pyenv (https://github.com/pyenv/pyenv)
- uv (rust based - python package manger)
    - _On macOS and Linux_
        - `curl -LsSf https://astral.sh/uv/install.sh | sh`
    - _windows_
        - coming soon
- Install the latest `backtesting.py` version 
    - `pip install git+https://github.com/kernc/backtesting.py.git`
        - This includes fixes for bokeh breaking when you are trying to plot
- TA Lib
    - `brew install ta-lib` - need ta-lib c library to work with ta-lib python
    - `pip install numpy==1.26.4` - need to ensure that we are using a numpy version lower then 2.0
- Jupyter Noteboks deps
    - `pip install ipykernel` for jupyter notebooks
- Install additional deps 
    - `pip install pandas ccxt`
    - Pandas dataframes
    - CCXT connection to various crypto exchanges

### Todos
- [x] [DONE] Update candle stick retrival to ensure that we are getting the full range that we are requesting
- [ ] Update strategy to also include stop loss, attempt to minimize risk as much as possible
- [ ] ADD RR using ATR make sure that it doesn't cause a infinite loop
- [ ] Figure out the best indicator for exiting a position with maximum upside
- [ ] Get yearly data 15m for BTC on all time frames to see which one is best
- [ ] Update candlestick data storage to parquet files
- [ ] Dynamic position size

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import talib
import os
import glob

%load_ext autoreload
%autoreload 2

## Strategy EMACross

EMA 20 X EMA 50

In [ ]:
class ImprovedEMACrossStrategy(Strategy):
    ema_fast = 10
    ema_slow = 30
    trade_size = 0.02
    take_profit_pct = 0.04
    stop_loss_pct = 0.02
    atr_period = 14
    rsi_period = 14
    rsi_overbought = 70
    rsi_oversold = 30
    atr_multiplier = 2
    trend_strength = 0.001

    def init(self):
        close = self.data.Close
        high = self.data.High
        low = self.data.Low

        self.ema1 = self.I(talib.EMA, close, timeperiod=self.ema_fast)
        self.ema2 = self.I(talib.EMA, close, timeperiod=self.ema_slow)
        self.atr = self.I(talib.ATR, high, low, close, timeperiod=self.atr_period)
        self.rsi = self.I(talib.RSI, close, timeperiod=self.rsi_period)
        self.entry_price = 0

    def next(self):
        price = self.data.Close[-1]

        # Dynamic take profit and stop loss based on ATR
        take_profit = self.atr[-1] * self.atr_multiplier
        stop_loss = self.atr[-1] * (self.atr_multiplier / 2)

        size = max(1, int((self.trade_size * self.equity) // price))

        # Check for take profit or stop loss
        if self.position.size != 0:
            pnl = price - self.entry_price
            if self.position.size > 0:
                if pnl >= take_profit or pnl <= -stop_loss:
                    self.position.close()
                    self.entry_price = 0
            elif self.position.size < 0:
                if pnl <= -take_profit or pnl >= stop_loss:
                    self.position.close()
                    self.entry_price = 0

        # Check for strong trend
        trend_strength = abs((self.ema1[-1] - self.ema2[-1]) / self.ema2[-1])

        if crossover(self.ema1, self.ema2) and trend_strength > self.trend_strength:
            if self.position.size < 0:
                self.position.close()
            if self.rsi[-1] < self.rsi_overbought:
                self.buy(size=size)
                self.entry_price = price
        elif crossover(self.ema2, self.ema1) and trend_strength > self.trend_strength:
            if self.position.size > 0:
                self.position.close()
            if self.rsi[-1] > self.rsi_oversold:
                self.sell(size=size)
                self.entry_price = price

### Load the saved candle stick data frame
_only run this cell when you want after you the have dynamically created the outputfile name_

In [ ]:
def find_parquet_file(timeframe, exchange, symbol="BTCUSDT"):
    # Base directory to search in
    base_directory = "./saved_candlestick_data/"

    # Full directory path including the exchange
    directory = os.path.join(base_directory, exchange)

    # Create the pattern to match the filename based on the exchange
    if exchange == "binance":
        pattern = f"{exchange}_{symbol}_{timeframe}_*.parquet"
    else:
        pattern = f"{symbol}_{timeframe}_*.parquet"

    # Search for files matching the pattern in the given directory
    matching_files = glob.glob(os.path.join(directory, pattern))

    if not matching_files:
        return None
    elif len(matching_files) > 1:
        # If multiple files are found, return the most recent one
        return max(matching_files, key=os.path.getctime)
    else:
        return matching_files[0]


# Usage example:
timeframe = "15m"
exchange = "binance"  # or "phemex"
symbol = "BTCUSDT"

filename = find_parquet_file(timeframe, exchange, symbol)
if filename:
    print(f"Found file: {filename}")
    df = pd.read_parquet(filename)
    # Now you can work with your dataframe
else:
    print(f"No file found for {symbol} with timeframe {timeframe} on {exchange}")

# Display the first few rows of the dataframe if it was loaded
if "df" in locals():
    print(df.head())
else:
    print("No dataframe to display")

### Run the strategy and plot it

In [ ]:
# Create the Backtest instance
bt = Backtest(
    df,
    ImprovedEMACrossStrategy,
    cash=100000,
    commission=0.002,
    exclusive_orders=True,
)

# Run the backtest
stats = bt.run()

# Print the results
print(stats)

# Plot the results
bt.plot()

## Debugging

In [ ]:
## check the size of the saved data
!ll